In [1]:
from math import log10
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pandas as pd
import os 

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader

from dataset.data_loader_RGB import *
from utils.pytorch_ssim import *

from SRGAN_model import Generator, Discriminator
from utils.loss import*

torch.manual_seed(1)
device = torch.device("cuda")

In [2]:
# SRGAN parameters

batch_size = 30
num_epochs = 10
lr = 0.0002
threads = 4
upscale_factor = 4

In [3]:
#img_path_low = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-56/train'
#img_path_ref = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-224/train'

img_path_low = '/home/jupyter/dataset/LR_56/train/'
img_path_ref = '/home/jupyter/dataset/HR/train/'

train_set = DatasetSuperRes(img_path_low, img_path_ref)
training_data_loader = DataLoader(dataset=train_set, num_workers=threads, batch_size=batch_size, shuffle=True)

In [4]:
netG = Generator(upscale_factor).to(device)
netD = Discriminator().to(device)

netG.weight_init(mean=0.0, std=0.2)
netD.weight_init(mean=0.0, std=0.2)

feature_extraction_model = initialize_senet50_2048()

generator_criterion = GeneratorLoss().to(device)
face_loss = FacePerceptionLoss(feature_extraction_model).to(device)
MSE_criterion = nn.MSELoss()

In [5]:
optimizerG = optim.Adam(netG.parameters(), lr=lr)
optimizerD = optim.Adam(netD.parameters(), lr=lr)
    
results = {'d_loss': [], 'g_loss': [], 'd_score': [],
           'g_score': [], 'psnr': [], 'ssim': []}

In [ ]:
# Let's pretrain the Generator
pre_train_epochs = 5
for epoch in range(1,pre_train_epochs+1):
    
    netG.train()
    
    for data, target in training_data_loader:
        data, target = data.to(device), target.to(device)
        
        netG.zero_grad()
        loss = MSE_criterion(netG(data), target)
        loss.backward()
        optimizerG.step()
    
    print('Finished pre-training {}/{}'.format(epoch,pre_train_epochs))

In [8]:
def train(epoch):
    
    running_results = {'batch_sizes': 0, 'd_loss': 0, 'g_loss': 0, 
                       'd_score': 0, 'g_score': 0}

    netG.train()
    netD.train()
    iteration = 0
    for data, target in training_data_loader:
        iteration+=1
        g_update_first = True
        batch_size = data.size(0)
        running_results['batch_sizes'] += batch_size

        ############################
        # (1) Update D network: maximize D(x)-1-D(G(z))
        ###########################
        real_img = Variable(target).to(device)
        z = Variable(data).to(device)
        
        fake_img = netG(z)

        netD.zero_grad()
        
        real_out = netD(real_img).mean()
        fake_out = netD(fake_img).mean()
        
        d_loss = 1 - real_out + fake_out
        d_loss.backward(retain_graph=True)
        
        optimizerD.step()

        ############################
        # (2) Update G network: minimize 1-D(G(z)) + Perception Loss + Image Loss + TV Loss
        ###########################
        netG.zero_grad()
        
        #mse_adversarial_content_loss = generator_criterion(fake_out, fake_img, real_img)
        face_perceptual_loss = face_loss(fake_img, real_img)
        
        g_loss = face_perceptual_loss
        
        g_loss.backward()

        fake_img = netG(z)
        fake_out = netD(fake_img).mean()

        optimizerG.step()

        # Loss for current batch before optimization 

        running_results['g_loss'] += g_loss.item() * batch_size
        running_results['d_loss'] += d_loss.item() * batch_size
        running_results['d_score'] += real_out.item() * batch_size
        running_results['g_score'] += fake_out.item() * batch_size

        print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, iteration, len(training_data_loader), d_loss.item()))

    print('[{}/{}] Loss_D: {:.4f} Loss_G: {:.4f} D(x): {:.4f} D(G(z)): {:.4f}'.format(
            epoch, num_epochs, 
            running_results['d_loss'] / running_results['batch_sizes'],
            running_results['g_loss'] / running_results['batch_sizes'],
            running_results['d_score'] / running_results['batch_sizes'],
            running_results['g_score'] / running_results['batch_sizes']))

    #netG.eval()

    batch_mse = ((fake_img - real_img) ** 2).data.mean()
    batch_ssim = ssim(fake_img, real_img).item()
    batch_psnr = 10 * log10(1 /batch_mse)

    out_path = 'results/'
    out_model_path = 'models/'
    
    if not os.path.exists(out_path):
        os.makedirs(out_path)    
    
    if not os.path.exists(out_model_path):
        os.makedirs(out_model_path)    

    # Save loss\scores\psnr\ssim
    results['d_loss'].append(running_results['d_loss'] / running_results['batch_sizes'])
    results['g_loss'].append(running_results['g_loss'] / running_results['batch_sizes'])
    results['d_score'].append(running_results['d_score'] / running_results['batch_sizes'])
    results['g_score'].append(running_results['g_score'] / running_results['batch_sizes'])
    results['psnr'].append(batch_psnr)
    results['ssim'].append(batch_ssim)

    if epoch % (num_epochs // 2) == 0:

        # Save model parameters
        torch.save(netG, out_model_path + 'SRGAN_x%d_epoch_%d.pth' % (upscale_factor, epoch))
        #torch.save(netD, 'netD_x%d_epoch_%d.pt' % (upscale_factor, epoch))

        data_frame = pd.DataFrame(
            data={'Loss_D': results['d_loss'], 'Loss_G': results['g_loss'], 'Score_D': results['d_score'],
                  'Score_G': results['g_score'], 'PSNR': results['psnr'], 'SSIM': results['ssim']},
            index=range(1, epoch + 1))
        data_frame.to_csv(out_path + 'SRGAN_x' + str(upscale_factor) + '_train_results.csv', index_label='Epoch')

In [9]:
num_epochs = 10

for epoch in range(1, num_epochs + 1):
    train(epoch)

AttributeError: 'float' object has no attribute 'backward'